In [2]:
import numpy as np
import skimage
from skimage.util import random_noise
import cv2

In [3]:
#this function was written by a computer vision course instructors 
def show_images(images, titles):
    #This function is used to show image(s) with titles by sending an array of images and an array of associated titles.
    # images[0] will be drawn with the title titles[0] if exists
    # You aren't required to understand this function, use it as-is.
    assert len(images) == len(titles)
    for title in titles:
        cv2.namedWindow(title, cv2.WINDOW_NORMAL)
    
    for title, img in zip(titles, images):
        cv2.imshow(title, img)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

# How to use show_images([list of images], [list of titles]) They must have the same length
# show_images([img1, img2], ['This is image 1', 'This is image 2'])

In [3]:
def apply_median_filter(image: np.array, kernel_size: int):
    denoised_image = cv2.medianBlur(skimage.img_as_uint(image),kernel_size)
    return denoised_image


In [4]:
def apply_negative_transform(img: np.array):
    imgTransformed = 255-img
    return imgTransformed

In [5]:
def linear_stretch(img: np.array):
    stretchedImg= (img - np.min(img))/(np.max(img)-np.min(img))
    return stretchedImg

In [6]:
def gamma_correction(img: np.array, c: float ,lamda : float ):
    img =skimage.img_as_float(img)
    correctedImg= c*(img**lamda)
    return correctedImg

In [7]:
def histogram_equalization(img: np.array):
    equalizedImg = cv2.equalizeHist(img)
    return equalizedImg


In [4]:
#this function was written by a computer vision course instructors 
def popularity_quantization(img: np.ndarray, number_of_colors: int) -> np.ndarray:
    assert number_of_colors > 1
    assert len(img.shape) == 3
               
    # Get the most popular color
    colors, counts = np.unique(img.reshape(-1, 3), return_counts = True, axis = 0)
    sorted_indicies = np.flip(np.argsort(counts))
    sorted_colors = colors[sorted_indicies]
    selected_colors = sorted_colors[:number_of_colors]
    # Get the nearest color for the pixel using Least square distance
    color_distance = np.sqrt(np.sum((np.expand_dims(img.reshape(-1, 3), axis=1) - selected_colors) ** 2, axis=2))
    indicies_of_the_least_distance = np.argmin(color_distance, axis=1)
    new_img = selected_colors[indicies_of_the_least_distance].reshape(img.shape)

    return new_img

In [8]:
# noisy images
img = cv2.imread('./assets/original.png', cv2.IMREAD_GRAYSCALE)
img = skimage.img_as_float(img)
n1 = random_noise(img, mode='s&p', clip=False, amount=0.02)
n2 = random_noise(img, mode='s&p', clip=False, amount=0.04)
n3 = random_noise(img, mode='s&p', clip=False, amount=0.08)
show_images([n1,n2,n3],['n1','n2','n3'])


In [9]:
# median filter
n1_denoised=apply_median_filter(n1,3)
n2_denoised=apply_median_filter(n2,3)
n3_denoised=apply_median_filter(n3,3)
show_images([n1_denoised,n2_denoised,n3_denoised],['denoise_n1','denoise_n2','denoise_n3'])

In [10]:
# negative transform test
img = cv2.imread('./assets/negative.png', cv2.IMREAD_GRAYSCALE)
imgTransformed=apply_negative_transform(img)
show_images([img,imgTransformed],['img','imgTransformed'])

In [11]:
# linear stretch test 
darkImg = cv2.imread('./assets/dark.png', cv2.IMREAD_GRAYSCALE)
badKidImg = cv2.imread('./assets/bad_kid.tif', cv2.IMREAD_GRAYSCALE)
darkImgLinearStretch = linear_stretch(darkImg)
badKidImgLinearStretch = linear_stretch(badKidImg)

show_images([darkImg,badKidImg,darkImgLinearStretch,badKidImgLinearStretch],['darkImg','badKidImg','darkImgLinearStretch','badKidImgLinearStretch'])

In [12]:
# histogram equalization test 
darkImg = cv2.imread('./assets/dark.png', cv2.IMREAD_GRAYSCALE)
badKidImg = cv2.imread('./assets/bad_kid.tif', cv2.IMREAD_GRAYSCALE)
darkImgEqualized = histogram_equalization(darkImg)
badKidImgEqualized = histogram_equalization(badKidImg)
show_images([darkImg,badKidImg,darkImgEqualized,badKidImgEqualized],['darkImg','badKidImg','darkImgEqualized','badKidImgEqualized'])

In [10]:
# popularity quantization algorithim test 
#500 seems to be a reasonable number to capture the image details 
imgWithColors = cv2.imread('./assets/image_with_colors.png')
imgWithColors = skimage.img_as_float(imgWithColors)
imgWithColorsQuantized = popularity_quantization(imgWithColors,500)
imgWithColorsQuantized = skimage.img_as_uint(imgWithColorsQuantized)
show_images([imgWithColors,imgWithColorsQuantized],['imgWithColors','imgWithColorsQuantized'])

the advantage of popularity algorithm over uniform quantization is that it distributes the existing colors to colors from whithin the image instead of distributing the colors to uniform ones that may not exist in the image and having a noticable conturing effect. But the popularity algorithms does not address colors with low popularity although theses colors may be significant. The median cut addresed that issue by representing each group of colors by a color from within the group so no color get ignored as in the case of the popularity algorithm  